In [ ]:
#Install to run R in notebook
!pip install rpy2==3.5.1

In [ ]:
#Run to run R in Notebook
%load_ext rpy2.ipython

In [ ]:
#Install R packages to retrieve and process groundwater data from USGS
%%R
install.packages("dataRetrieval")
install.packages("tidyverse")
install.packages("lubridate")
library(dataRetrieval)
library(tidyverse)
library(lubridate)

In [ ]:
#Install Python Packages for processing and plotting data
import datetime
import os
import requests
import time

!pip install --quiet geopandas
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# Set Up for API Calls
#Prep for API Call
# Set root URL for API requests
root_url = 'https://api.climateengine.org/'
# Authentication info for the API (INSERT YOUR OWN KEY)
headers = {'Authorization': 'INSERT YOUR API KEY HERE'}

In [ ]:
#SiteNumber
siteNum = '380128105484401'

#Coordinates of Site
coordinates = '[[-105.812,38.024]]'

In [ ]:
#API Request for Data
#Select endpoint that exports a map of values to a Google Cloud storage bucket
Endpoint = 'timeseries/interannual/points'

#Set up parameters dictionary for API call
ndviParams = {
    'dataset': 'LANDSAT_SR',
    'variable': 'NDVI',
    'start_day': '01',
    'end_day': '31',
    'start_month':'06',
    'end_month':'08',
    'start_year': '1986',
    'end_year': '2023',
    'coordinates': coordinates,
    'area_reducer': 'mean',
    'temporal_statistic': 'median'
}

# Send API request
ndviR = requests.get(root_url + Endpoint, params=ndviParams, headers=headers, verify=False)
ndviExport_response = ndviR.json()

#Set up parameters dictionary for API call
precipParams = {
    'dataset': 'GRIDMET',
    'variable': 'pr',
    'start_day': '01',
    'end_day': '30',
    'start_month':'10',
    'end_month':'09',
    'start_year': '1986',
    'end_year': '2023',
    'coordinates': coordinates,
    'area_reducer': 'mean',
    'temporal_statistic': 'sum'
}

# Send API request
precipR = requests.get(root_url + Endpoint, params=precipParams, headers=headers, verify=False)
precipExport_response = precipR.json()

In [ ]:
#Process RAP Data
#response (may need to unpack with [] around timeseries the first time)
[timeseries1] = ndviExport_response
[timeseries2] = precipExport_response

#Select Data
data1 = timeseries1['Data']
data2 = timeseries2['Data']

#Convert Data to Dataframe
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

#Set Date
date = pd.to_datetime(df1['Year'])

#Assign Values
value1 = df1['NDVI']
value2 = df2['pr (mm)']

In [ ]:
#Request Well Data
%%R

#Define Parameters
siteNo <- "380128105484401"
start.date <- "1986-01-01"
end.date <- "2022-12-31"

#Create Request
siteOutput <- readNWISgwl(siteNumbers = siteNo,
                       startDate = start.date,
                       endDate = end.date)

#Make Request
siteOutput <- renameNWISColumns(siteOutput)

#Process Data for Plotting
siteOutput_Proc <- siteOutput %>%
  drop_na(lev_va) %>%
  select(lev_dt,lev_va) %>%
  mutate(year = year(ymd(lev_dt)), month = month(ymd(lev_dt)), day = day(ymd(lev_dt))) %>%
  group_by(year) %>%
  filter(month > 5 & month <9) %>%
  summarize(JJAMean = mean(lev_va))

print(siteOutput_Proc, n=39)

In [ ]:
#Set data to variable
valueElse = %R siteOutput_Proc$JJAMean

#Fill any missing years
#valueElse = np.insert(valueElse,0,np.nan) #86

#Convert to Dataframe
dfElse = pd.DataFrame(valueElse)
dfElse

In [ ]:
from sys import ps2
import matplotlib.dates as mdates

#Calculate Trendlines

#NDVI Trend
x1 = mdates.date2num(date)
z1 = np.polyfit(x1, value1, 1)
p1 = np.poly1d(z1)

#Precipitation Trend
x2 = mdates.date2num(date)
z2 = np.polyfit(x2, value2, 1)
p2 = np.poly1d(z2)

#Groundwater Trend
x3 = mdates.date2num(date)
z3 = np.polyfit(x3, valueElse, 1)
p3 = np.poly1d(z3)


In [ ]:
# create figure and axis objects with subplots()
fig,ax = plt.subplots(2,figsize = (18,8))

# make a plot
ax[0].scatter(date, value1, color="green", label = 'NDVI', s=10)
ax[1].scatter(date, value2, color="purple", label = 'Precipitation', s=10)

ax[0].plot(date, value1, color="green", label = 'NDVI')
ax[1].plot(date, value2, color="purple", label = 'Precipitation')

ax[0].plot(x1, p1(x1),  linestyle = "dotted", color = "green", alpha = 0.75)
ax[1].plot(x2, p2(x2),  linestyle = "dotted", color = "purple", alpha = 0.75)

#Change Y1 Scale
ax[0].autoscale()
ax[1].autoscale()

#Add x-axis label
ax[0].set_xlabel("Year", fontsize = 14)
ax[1].set_xlabel("Year", fontsize = 14)

#Add first y-axis label
ax[0].set_ylabel("NDVI", color="green", fontsize=14)
ax[1].set_ylabel("Precipitation (mm)", color="purple", fontsize=14)

# twin object for two different y-axis on the sample plot
ax2=ax[0].twinx()
ax3=ax[1].twinx()

# make a plot with different y-axis using second axis object
ax2.plot(date, valueElse, color="blue")
ax2.scatter(date, valueElse, color="blue", s=10)
ax2.plot(x3, p3(x3),  linestyle = "dotted", color = "blue", alpha = 0.75)
ax2.margins(x=0)
ax3.plot(date, valueElse, color="blue")
ax3.scatter(date, valueElse, color="blue", s=10)
ax3.plot(x3, p3(x3),  linestyle = "dotted", color = "blue", alpha = 0.75)
ax3.margins(x=0)

#Change Y1 Scale
ax2.autoscale()
ax3.autoscale()

#Reverse Axis
ax2.invert_yaxis()
ax3.invert_yaxis()

#Add Second y-axis Label
ax2.set_ylabel("Depth to Groundwater (ft)",color="blue",fontsize=14)
ax3.set_ylabel("Depth to Groundwater (ft)",color="blue",fontsize=14)

#Add Title
fig.suptitle('Well SiteNo' + " " + siteNum, fontsize=20)

#Export Graph
plt.savefig('NDVI-PPT-GW_Well' + siteNum + '.png', bbox_inches='tight')

#Show Graph
plt.show()